In [1]:
import sys
import random
import pickle
import argparse
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pprint import pprint

sys.path.append("../")

from utils.parse_arxiv import *
from keyword_extractor import *
from workloads.query_gen import *

In [2]:
file = open('../data/filtered_data.pickle', 'rb')
data = pickle.load(file)
file.close()

data.reset_index(drop=True, inplace=True)

# data.head(3)

In [3]:
tmp = dict(data.iloc[0])
tmp['abstract'].replace('\n', ' ')

'  The shape of the hadronic form factor f+(q2) in the decay D0 --> K- e+ nue has been measured in a model independent analysis and compared with theoretical calculations. We use 75 fb(-1) of data recorded by the BABAR detector at the PEPII electron-positron collider. The corresponding decay branching fraction, relative to the decay D0 --> K- pi+, has also been measured to be RD = BR(D0 --> K- e+ nue)/BR(D0 --> K- pi+) = 0.927 +/- 0.007 +/- 0.012. From these results, and using the present world average value for BR(D0 --> K- pi+), the normalization of the form factor at q2=0 is determined to be f+(0)=0.727 +/- 0.007 +/- 0.005 +/- 0.007 where the uncertainties are statistical, systematic, and from external inputs, respectively. '

In [4]:
data.iloc[100]['abstract']

'  A network identification by deconvolution (NID) method is applied to the\nthermal transient response of packaged and unpackaged microcoolers. A thin film\nresistor on top of the device is used as the heat source and the temperature\nsensor. The package and the bonding thermal resistances can be easily\nidentified by comparing structure functions. High-speed coplanar probes are\nused to achieve a short time resolution of roughly 100ns in the transient\ntemperature response. This is used to separate the thermal properties of the\nthin film from the substrate. The obtained thermal resistances of the buffer\nlayer and Silicon substrate are consistent with the theoretical calculations.\nIn order to estimate the superlattice thermal resistance and separate it from\nthe thin SiNx layer deposited underneath the thin film resistive sensor, an\norder of magnitude faster thermal transient response is needed.\n'

In [5]:
qd = QueryTemplate()
qd.print_info()

title       :None
abstract    :None
author      :[]
year        :None
categories  :None
keywords    :[]
journal     :None


In [6]:
qd.parse_info(tmp)
qd.print_info()

title       :Measurement of the Hadronic Form Factor in D0 --> K- e+ nue Decays
abstract    :The shape of the hadronic form factor f+(q2) in the decay D0 --> K- e+ nue has been measured in a model independent analysis and compared with theoretical calculations. We use 75 fb(-1) of data recorded by the BABAR detector at the PEPII electron-positron collider. The corresponding decay branching fraction, relative to the decay D0 --> K- pi+, has also been measured to be RD = BR(D0 --> K- e+ nue)/BR(D0 --> K- pi+) = 0.927 +/- 0.007 +/- 0.012. From these results, and using the present world average value for BR(D0 --> K- pi+), the normalization of the form factor at q2=0 is determined to be f+(0)=0.727 +/- 0.007 +/- 0.005 +/- 0.007 where the uncertainties are statistical, systematic, and from external inputs, respectively.
author      :['TheBABARCollaboration', 'B.Aubert', 'etal']
year        :2015
categories  :['hep-ex']
keywords    :['present world average value', 'corresponding decay branch

In [7]:
qd.generate_query(title=True)

'find papers titled < Measurement of the Hadronic Form Factor in D0 --> K- e+ nue Decays >'

In [8]:
for i in range(10):
    print(qd.generate_query())

find papers related to < data recorded > published at < Phys.Rev.D76:052005,2007 >
find papers written by < TheBABARCollaboration > from year 2015 about < hep-ex > related to < 0 >
find papers written by < TheBABARCollaboration > from year 2015 about < hep-ex > related to < theoretical calculations >
find papers written by < etal > about < hep-ex > published at < Phys.Rev.D76:052005,2007 >
find papers about < hep-ex > published at < Phys.Rev.D76:052005,2007 >
find papers about < hep-ex > published at < Phys.Rev.D76:052005,2007 >
find papers related to < 0 >
find papers written by < B.Aubert > from year 2015 published at < Phys.Rev.D76:052005,2007 >
find papers written by < B.Aubert > from year 2015 about < hep-ex > published at < Phys.Rev.D76:052005,2007 >
find papers written by < TheBABARCollaboration > from year 2015 related to < theoretical calculations > published at < Phys.Rev.D76:052005,2007 >
